##### Imports

In [107]:
# # # # # # Imports
from urllib.request import urlopen
import json
import urllib
import ipywidgets as widgets
from IPython.display import display, clear_output, Image
import re

##### Ideen/ Weiterentwicking
1. Prio

2. Prio
- Buttons auf ENTER reagieren
- bei Mehrfachantworten (Grundstück durchsticht mehrere Werte / Features)? ev einfach weiteren Kontext zu den Antworten angeben


##### Functions

In [108]:
# # # # # # Funktionen
# # # # Adressvorshläge basierend auf Eingabe generieren
def get_adr_lst(eingabe_suchwort):

        adr_suche = eingabe_suchwort
        adr_suche_corr = '%20'.join(un_um_laut(adr_suche).split(' ')) # Umlaute entfernen und Leerschläge ersetzen 
        adr_suche_url = f'https://www.metawarehouse.apps.be.ch/rpc/oereb_search?searchtext={adr_suche_corr}' # URL für Koordinatenabfrage
        # URL öffnen und JSON laden
        response = urlopen(adr_suche_url)
        data_json = json.loads(response.read())
        adr_list = [] # Liste gefundener Adressen generieren
        if data_json:
            for i in data_json:
                adr_list.append(i['label'])
        else:
            pass

        return adr_list, data_json

# # Umlaute ausschreiben
def un_um_laut(text):
    next = text.replace('ü', 'ue').replace('ö', 'oe').replace('ä', 'ae')
    next=next.replace('Ü', 'Ue').replace('Ö', 'Oe').replace('Ä', 'Ae')
    next=next.replace('è','e').replace('È','E').replace('é','e').replace('É','E').replace('â','a').replace('Â','A')
    return(next)

# # # #  Ausgabe für den Durchstich generieren
def Durchstich_output(theme, place, json = None):
    parcelinfo_url = get_parcelinfo(place, json=json)
    antwort = search_parcelinfo(theme = theme, parc_inf = parcelinfo_url)
    return antwort

# # parcelinfo abrufen 
def get_parcelinfo(place, json = None):
    if not json:
        egrid = place
        parcelinfo_url = 'https://www.' + f'metawarehouse-test.apps.be.ch/rpc/hackdays_parcelinfo_egrid?egrid={egrid}'
    else:
        adr_suche = place
        x = [i['x'] for i in json if i['label'] == adr_suche][0]
        y = [i['y'] for i in json if i['label'] == adr_suche][0]
        
        parcelinfo_url = 'https://www.' + f'metawarehouse-test.apps.be.ch/rpc/hackdays_parcelinfo_xy?x={x}&y={y}'
    return parcelinfo_url


# # Parzelleninfos nach themen durchsuchen und Liste für Ausgabe generieren
def search_parcelinfo(theme, parc_inf):
    response = urlopen(parc_inf)
    data_json = json.loads(response.read())
    nat_gef = [i for i in data_json if theme.lower() in i['topic'].lower()]
    if 'x=' and 'y=' in parc_inf:
        x = parc_inf.split('x=')[-1][:7]
        y = parc_inf.split('y=')[-1][:7]
        egrid = None # aus Thema Liegenschaften holen, um Parzellengrenze zu highlighten?
    else:
        egrid = parc_inf[-14:]
        x,y = None, None 
    
    
    topic_descr = [[
                    i['topic'],
                    i['description'],
                    i['metadata'],
                    ( # Links erstellen für layer/ grp mit x,y oder EGRID
                    f'https://www.topo.apps.be.ch/pub/map/?center={x},{y}&addcrosshair=true&scale=1500&lang=de&gpk={i["gpr_code"]}_GPK&legend=True'
                    if x and i['object_type'] == "geoproduct"
                    else f'https://www.topo.apps.be.ch/pub/map/?center={x},{y}&addcrosshair=true&scale=1500&lang=de&layers={i["gpr_code"]}_{i["code"]}&legend=True'
                    if x
                    else f'https://www.topo.apps.be.ch/pub/map/?DIPANU_DIPANUF_KMGDM1=DIPANUF_EGRID={egrid}&addcrosshair=true&scale=1500&lang=de&gpk={i["gpr_code"]}_GPK&legend=True'
                    if egrid and i['object_type'] == "geoproduct"
                    else f'https://www.topo.apps.be.ch/pub/map/?DIPANU_DIPANUF_KMGDM1=DIPANUF_EGRID={egrid}&addcrosshair=true&scale=1500&lang=de&layers={i["gpr_code"]}_{i["code"]}&legend=True'
                    ),
                    #filter for Egrid? --> use either egrid or xy for all possible key: Liegenschaften
                    i["object_type"],
                    i["code"],
                    i["gpr_code"]]
                    for i in nat_gef
                    ]

    return topic_descr


##### Widgets

In [109]:
# # # # # # Widgets
# # Styles
button_color = '#f3e5ce'  # Button background color
text_color = '#000000'    # Text color
hover_color = '#ebd3ae'   # Hover color

# Apply the colors using inline CSS in an HTML widget for hover effect
style_button = widgets.HTML(
    value=f"""
    <style>
        .custom-button {{
            background-color: {button_color} !important;
            color: {text_color} !important;
            border: none;
            font-size: 14px;
            
        }}
        .custom-button:hover {{
            background-color: {hover_color} !important;
        }}
        .custom-button:focus {{
            outline: 1px solid #ea161f !important; /* Keep blue outline (you can change it if desired) */
        }}
        .custom-button:active {{
            background-color: {hover_color} !important; /* Ensure the color stays consistent when active */
            outline: 1px solid #ea161f !important; /* Keep the custom blue outline */
        }}

        /* CUSTOM dropdown*/
        .custom-dropdown select:focus {{
            outline: 1px solid #ea161f !important; /* Keep blue outline (you can change it if desired) */
        }}
        .custom-dropdown select:active {{
            outline: 1px solid #ea161f !important; /* Keep the custom blue outline */
        }}

        /*CUSTOM input*/
        .custom-input input:hover {{
             outline: 1px solid #ea161f !important; /* Keep blue outline (you can change it if desired) */
        }}
        .custom-input input:focus {{
            outline: 1px solid #ea161f !important; /* Keep blue outline (you can change it if desired) */
        }}

    </style>
    """
)



In [110]:
# # # Widget Creation
# Suchthema definieren
theme = widgets.Text(value = '', 
                    placeholder='Thema (ohne Auswalh = alle)', 
                    ).add_class('custom-input')

# Dropdown für Themenauswahl erstellen
thm_dropdown = widgets.Dropdown( 
    options=[],
    disabled=False
).add_class('custom-dropdown')        

# # Textfeld für Suchen nach EGRID
egrid = widgets.Text(placeholder='Gebäudeidentifikator (EGRID)').add_class('custom-input') # EGRID Eingabe

# # Adresseingabe
# Textfeld erstellen
adr_textfeld = widgets.Text( # Textfeld für erste Eingabe
    placeholder = 'Adresse',
    disabled = False
).add_class('custom-input')

# Dropdown für Adressauswahl erstellen
adr_dropdown = widgets.Dropdown(
    options=[],
    disabled=False
).add_class('custom-dropdown')

adr_such_btn = widgets.Button(description='Adresse suchen').add_class('custom-button') # Button, Adresse Suchen
thm_such_btn = widgets.Button(description='Thema suchen').add_class('custom-button') # Button, Thema Suchen
refresh_btn = widgets.Button(description="Neu").add_class('custom-button') # Button, Neue Adresse eingeben
thm_refresh_btn = widgets.Button(description="Neu").add_class('custom-button') # Button, Neue Adresse eingeben

# # Durchstich Suche
button_send = widgets.Button(
                description='Suche',
                tooltip='Send'
                # style={'description_width': 'initial'}
            ).add_class('custom-button')

# Horizontale Linie für Darstellung
horizontal_line = widgets.Box(layout=widgets.Layout(
    width='100%',  
    border = 'solid 1px black'  

))


###### VBox Adressuche

In [111]:
# # # # # # Suchfunktionen
# # Adresse
def adr_such_fct(b): # Adresssuche
    global koord_json

    input_value=adr_textfeld.value
    if input_value:
        sugg, koord_json = get_adr_lst(input_value)
        if sugg:
            adr_dropdown.options = sugg
            vbox.children = [adr_dropdown, refresh_btn]
        else:
            adr_textfeld.value = ''
            adr_textfeld.placeholder='Adresse nicht gefunden'
            vbox.children = [adr_textfeld, style_button,adr_such_btn]
    else:
        vbox.children = [adr_textfeld, style_button,adr_such_btn]

def refr_cell(b): # Suchfeld aktualisieren
    adr_textfeld.value = ""  
    adr_dropdown.value = None
    vbox.children = [adr_textfeld, style_button,adr_such_btn]

adr_such_btn.on_click(adr_such_fct)
refresh_btn.on_click(refr_cell)

vbox = widgets.HBox([adr_textfeld, style_button,adr_such_btn])

###### VBox Suche

In [112]:
# # # Suche Durchstich
# # Tabelle für Output
headers = ['Thema','Wert','Detailinfos','Kartenansicht']
levels = {
    'Synoptische Gefahrenkarte nicht vollständig vorhanden':-1,
    'Keine Gefährdung bekannt': 0, 
    'Restgefährdung':1, 
    'Geringe Gefährdung':2, 
    'Mittlere Gefährdung':3, 
    'Erhebliche Gefährdung':4}

def text_out(Llist, name):

    def unique_highest_value(list):
        first_list = [i[1] for i in list]
        return max([levels[i] for i in set(first_list)])

    def find_keys_by_value(dictionary, list):
        risk_level = [key for key, value in dictionary.items() if value == unique_highest_value(list)][0]
        return [i for i in list if i[1] == risk_level][0]


    max_lvl_ngf = find_keys_by_value(levels, [i for i in Llist if name in i[0]])
    # max_lvl = unique_highest_value(levlist)
    return max_lvl_ngf

def out_table(Llist, headers):
    # HTML Tabelle erstellen
    table_html = "<table style='border-collapse: collapse; width: 100%;'>"#font-family: Helvetica;

    # Kopfzeile hinzufügen
    table_html += "<tr>"
    for header in headers:
        table_html += f"<th style='border: 1px solid black; padding: 8px; background-color: #f2f2f2; font-weight: bold; text-align: left;'>{header}</th>"
    table_html += "</tr>"

    seen_values = set()

    # Zeilen hinzufügen
    for sublist in Llist:
        sublist_out = sublist[:-3]
        table_html += "<tr>"
        # Hier implementieren, falls layer und GPR andere Darstellungen in Tabelle haben sollen
        # if sublist[-3] == 'layer':
        if sublist_out[0] in seen_values:
            n_sublist = sublist_out
            n_sublist[0] = ''
            for item in n_sublist:
                table_html += f"<td style='border: 1px solid black; padding: 8px; background-color: #FFFFFF; text-align: left;'>{item}</td>"
        else:
            for item in sublist_out:
                table_html += f"<td style='border: 1px solid black; padding: 8px; background-color: #FFFFFF; text-align: left;'>{item}</td>" 
            seen_values.add(sublist_out[0])  # Add the new value to the set       


        table_html += "</tr>"

    table_html += "</table>"
    return table_html

In [113]:
# antwort = Durchstich_output(thema, 'CH773517864624')
# antwort

In [114]:
# # Durchstich Suche
output = widgets.Output(layout={'font-size': '20px'})
output_str = ''
thema = 'Naturgefahr'
sub_thema = ['Lawine', 'Sturz', 'Wasser', 'Rutschung', 'Einsturz/Absenkung']
def on_button_clicked(event):

    with output:
        clear_output()
        if adr_textfeld.value or adr_dropdown.value:
            if not adr_dropdown.value:
                adr_such_fct(None)
            try:
                output_text = (
                    f"<h3>Informationen zu Naturgefahren an der Adresse {adr_dropdown.value}:</h3><br>" #style='font-family:Helvetica;'
                    "<p>Da die Informationen der Parzellenfläche entsprechen, können für bestimmte Themen mehrere Werte ausgegeben werden.</p>"
                    )
                antwort = Durchstich_output(thema, adr_dropdown.value, koord_json) 
                
                if antwort:
                    # URL zu Link
                    def replace_urls(text):
                        text = re.sub(r'(https://www\.agi\.dij\.be\.ch\S+)', r"<a href='\1' target='_blank'>Geoprodukt</a>", text)
                        text = re.sub(r'(https://www\.topo\.apps\.be\.ch\S+)', r"<a href='\1' target='_blank'>Karte</a>", text)
                        return text

                    # Apply the function to each sublist in 'antwort'
                    antwort = [[replace_urls(text) for text in sublist] for sublist in antwort]
                    # print(antwort)
                    synopt = text_out(antwort, 'Synoptische')
                    paragraph = (
                        "<h3>Gesamtgefährdung (Synoptische Gefahrengebiete)</h3>"# style='font-family:Helvetica;'
                        f"<p>Die Gesamtgefährdung an der von Ihnen gewählten Adresse wird mit folgendem Wert eingestuft: <strong>{synopt[1]}</strong><br>"
                        "Dies ist die zur Beurteilung von Baugesuchen massgebende Gefahrenstufe. Sie entsteht durch die Überlagerung der Gefahrtenkarten aller Hauptprozesse. "
                        "Die höchste Gefahrenstufe ist jeweils massgebend. "
                        "Diese Einstufung ist relevant für Standorte innerhalb von Siedlungsgebieten. Ausserhalb dieses Bereichs ist die Gefahrenhinweiskarte massgebend (Link?). "
                        f"Schauen sie sich die {synopt[3]} für Ihren Standort an. "
                        f"Genauere Informationen zu den zugrundeliegenden Daten können der Detailseite für das {synopt[2]} entnommen werden.<p>"
                    )
                    
                    Hauptprozesse = "<h3>Hauptprozesse</h3>"# style='font-family:Helvetica;
                    
                    relev_natgef, irrelev_natgef = [], []
                    for entry in sub_thema:
                        t_search = text_out(antwort, entry)
                        t_search[0] = t_search[0].split(' ')[-1]
                        # Steuern, wie Gefahren mit keiner Gefährdung, oder keinen Daten angezeigt werden sollen
                        if levels[t_search[1]] > 0:
                            relev_natgef.append(t_search)
                        else:
                            irrelev_natgef.append(t_search)
                                                
                    if relev_natgef:
                        sort_relev_natgef = sorted(relev_natgef, key=lambda sublist: levels.get(sublist[1],0), reverse= True)
                        p_relev_natgef="<p>Folgende Hauptprozesse sind am gewählten Standort relevant. Sie sind nach absteigeder Gefahrenstufe sortiert:"
                        table_natgef = out_table(sort_relev_natgef, ['Prozess', 'Gefahrenstufe', 'Detailinfos', 'Kartenansicht'])
                        p_relev_natgef += table_natgef
                    else:
                        sort_relev_natgef = []

                    if irrelev_natgef:
                        if len(irrelev_natgef) > 2:
                            aufzählung = ', '.join([i[0] for i in irrelev_natgef[:-1]])+' und '+irrelev_natgef[-1][0]
                        elif len(irrelev_natgef) == 2:
                            aufzählung = ' und '.join([i[0] for i in irrelev_natgef])
                        else:
                            aufzählung = [i[0] for i in irrelev_natgef][0]
                        p_irrelev_natgef= f"<p>Für die Hauptprozesse {aufzählung} ist am gewählten Standort keine Gefährdung bekannt.</p>"

                    Regelungen="<a href='https://www.naturgefahren.sites.be.ch/de/start/themen/bauen-in-gefahrengebieten/umsetzung-gemeinden.html' target='_blank'>Regelungen</a>"#textimage_2110739074' # Mehr erfahren zur gefahrenkarte und Gefahrenstufe
                    bauen_gefahr="<a href='https://www.naturgefahren.sites.be.ch/de/start/themen/bauen-in-gefahrengebieten.html' target='_blank'>Bauen in Gefahrengebieten</a>"
                    Gefahreninformationen = "<a href='https://www.naturgefahren.sites.be.ch/de/start/themen/gefahreninformationen.html' target='_blank'>Gefahreninformationen</a>" # Auch Infos ausserhalb Siedlungszone
                    Naturgefahren="<a href='https://www.naturgefahren.sites.be.ch/de/start.html' target='_blank'>Naturgefahren</a>"
                    p_abschluss = (
                        "<p>Da die gegebenen Informationen für die ganze Parzelle gelten, lohnt sich ein Blick auf die oben verlinkten Karten. "
                        "Möglicherweise sind Gebäude ausserhalb der gefährdeten Bereiche. "
                        f"Informieren Sie sich weiter über {Naturgefahren} im Kanton Bern:<br>"
                        "<ul>"
                        f"<li>Für das {bauen_gefahr} gibt es bestimmte {Regelungen}, um Schäden und Leid zu vermeiden und Folgekosten zu minimieren. </li>" 
                        f"<li>Weitere {Gefahreninformationen}.</li>"
                        "</ul></p>"
                    )
                    # Beschreibungen für die spez Naturgefahren (absteigend nach vorhandener maximaler Gefahrenstufe)
                    

                    if paragraph:
                        display(widgets.HTML(paragraph))
                    display(widgets.HTML(Hauptprozesse))
                    if sort_relev_natgef:
                        display(widgets.HTML(p_relev_natgef))
                    if irrelev_natgef:
                        display(widgets.HTML(p_irrelev_natgef))
                    display(widgets.HTML(p_abschluss))

                else:
                    display(widgets.HTML('<p>Keine Informationen gefunden Entweder gibt es am Gewählten Ort keine Daten, oder das Thema ist dort nicht relevant.</p>'))
                

            except urllib.error.HTTPError:
                display(widgets.HTML('<p> Für die eingegebene Adresse wurde keine Parzelle gefunden!</p>'))
            except Exception as e:
                display(widgets.HTML(f'<p>Ein unerwarteter Fehler ist aufgetreten: {str(e)}. Überprüfen Sie die Adresseingabe</p>'))   

            
        elif egrid.value:
            try:

                output_text = (
                    f"<h3>Informationen zu Naturgefahren für das Grundstück {egrid.value}:</h3>"#style='font-family:Helvetica;'
                    "<p>Da die Informationen der Parzellenfläche entsprechen, können für bestimmte Themen mehrere Werte ausgegeben werden.</p>"
                )
                
                antwort = Durchstich_output(thema, egrid.value)
                if antwort:
                    # URL zu Link
                    def replace_urls(text):
                        text = re.sub(r'(https://www\.agi\.dij\.be\.ch\S+)', r"<a href='\1' target='_blank'>Geoprodukt</a>", text)
                        text = re.sub(r'(https://www\.topo\.apps\.be\.ch\S+)', r"<a href='\1' target='_blank'>Karte</a>", text)
                        return text

                    antwort = [[replace_urls(text) for text in sublist] for sublist in antwort]

                    synopt = text_out(antwort, 'Synoptische')
                    paragraph = (
                        "<h3>Gesamtgefährdung (Synoptische Gefahrengebiete)</h3>"# style='font-family:Helvetica;'
                        f"<p>Die Gesamtgefährdung an der von Ihnen gewählten Adresse wird mit folgendem Wert eingestuft: <strong>{synopt[1]}</strong><br>"
                        "Dies ist die zur Beurteilung von Baugesuchen massgebende Gefahrenstufe. Sie entsteht durch die Überlagerung der Gefahrtenkarten aller Hauptprozesse. "
                        "Die höchste Gefahrenstufe ist jeweils massgebend. "
                        "Diese Einstufung ist relevant für Standorte innerhalb von Siedlungsgebieten. Ausserhalb dieses Bereichs ist die Gefahrenhinweiskarte massgebend (Link?). "
                        f"Schauen sie sich die {synopt[3]} für Ihren Standort an. "
                        f"Genauere Informationen zu den zugrundeliegenden Daten können der Detailseite für das {synopt[2]} entnommen werden.<p>"
                    )

                    Hauptprozesse = "<h3>Hauptprozesse</h3>"# style='font-family:Helvetica;
                    
                    relev_natgef, irrelev_natgef = [], []
                    for entry in sub_thema:
                        t_search = text_out(antwort, entry)
                        t_search[0] = t_search[0].split(' ')[-1]
                        # Steuern, wie Gefahren mit keiner Gefährdung, oder keinen Daten angezeigt werden sollen
                        if levels[t_search[1]] > 0:
                            relev_natgef.append(t_search)
                        else:
                            irrelev_natgef.append(t_search)

                    if relev_natgef:
                        sort_relev_natgef = sorted(relev_natgef, key=lambda sublist: levels.get(sublist[1],0), reverse= True)
                        p_relev_natgef="<p>Folgende Hauptprozesse sind am gewählten Standort relevant. Sie sind nach absteigeder Gefahrenstufe sortiert:"
                        table_natgef = out_table(sort_relev_natgef, ['Prozess', 'Gefahrenstufe', 'Detailinfos', 'Kartenansicht'])
                        p_relev_natgef += table_natgef
                    else:
                        sort_relev_natgef = []

                    if irrelev_natgef:
                        if len(irrelev_natgef) > 2:
                            aufzählung = ', '.join([i[0] for i in irrelev_natgef[:-1]])+' und '+irrelev_natgef[-1][0]
                        elif len(irrelev_natgef) == 2:
                            aufzählung = ' und '.join([i[0] for i in irrelev_natgef])
                        else:
                            aufzählung = [i[0] for i in irrelev_natgef][0]
                        p_irrelev_natgef= f"<p>Für die Hauptprozesse {aufzählung} ist am gewählten Standort keine Gefährdung bekannt.</p>"

                    Regelungen="<a href='https://www.naturgefahren.sites.be.ch/de/start/themen/bauen-in-gefahrengebieten/umsetzung-gemeinden.html' target='_blank'>Regelungen</a>"#textimage_2110739074' # Mehr erfahren zur gefahrenkarte und Gefahrenstufe
                    bauen_gefahr="<a href='https://www.naturgefahren.sites.be.ch/de/start/themen/bauen-in-gefahrengebieten.html' target='_blank'>Bauen in Gefahrengebieten</a>"
                    Gefahreninformationen = "<a href='https://www.naturgefahren.sites.be.ch/de/start/themen/gefahreninformationen.html' target='_blank'>Gefahreninformationen</a>" # Auch Infos ausserhalb Siedlungszone
                    Naturgefahren="<a href='https://www.naturgefahren.sites.be.ch/de/start.html' target='_blank'>Naturgefahren</a>"
                    p_abschluss = (
                        "<p>Da die gegebenen Informationen für die ganze Parzelle gelten, lohnt sich ein Blick auf die oben verlinkten Karten. "
                        "Möglicherweise sind Gebäude ausserhalb der gefährdeten Bereiche. "
                        f"Informieren Sie sich weiter über {Naturgefahren} im Kanton Bern:<br>"
                        "<ul>"
                        f"<li>Für das {bauen_gefahr} gibt es bestimmte {Regelungen}, um Schäden und Leid zu vermeiden und Folgekosten zu minimieren. </li>" 
                        f"<li>Weitere {Gefahreninformationen}.</li>"
                        "</ul></p>"
                    )
                    # Beschreibungen für die spez Naturgefahren (absteigend nach vorhandener maximaler Gefahrenstufe)
                    if paragraph:
                        display(widgets.HTML(paragraph))
                    display(widgets.HTML(Hauptprozesse))
                    if sort_relev_natgef:
                            display(widgets.HTML(p_relev_natgef))
                    if irrelev_natgef:
                        display(widgets.HTML(p_irrelev_natgef))
                    display(widgets.HTML(p_abschluss))  
                    
                else:
                    display(widgets.HTML('<p>Keine Informationen gefunden Entweder gibt es am Gewählten Ort keine Daten, oder das Thema ist dort nicht relevant.</p>'))

            except urllib.error.HTTPError:
                display(widgets.HTML('<p> Für die eingegebene Adresse wurde keine Parzelle gefunden!</p>'))
            # except NameError:
            #     display(widgets.HTML('<p>Fehlerhafte Suche. Möglicherweise hat die EGRID Eingabe nicht das richtige Format</p>'))
            except Exception as e:
                display(widgets.HTML(f'<p>Ein unerwarteter Fehler ist aufgetreten: {str(e)}. Überprüfen Sie die EGRID Eingabe.</p>'))
  
            # # # # #

        else:
             output_text = f"<p>Keine Informationen gefunden Entweder gibt es am Gewählten Ort keine Daten, oder das Thema ist dort nicht relevant. <br>Haben Sie eine Adresse oder EGRID-Nummer eingegeben?</p>"
             display(widgets.HTML(output_text))
        

button_send.on_click(on_button_clicked)
vbox_result = widgets.VBox([style_button,
                            button_send, 
                            output
                            ])


##### Interface
###### Villa Morillon: CH382646358083 Reiterstrasse 11: CH743546874207 Heckenweg: CH113595468791 Bümpliz: CH868946351291

In [115]:
# # # # # # Darstellung
display(
    widgets.HBox([
        widgets.Image(
            value=Image('K:\Benutzer\m4bc_Jos\Portal for ArcGIS\Web_Logos\Kanton_Bern_RGB\png\Kein_Schutzraum\Kanton_Bern_KS_RGB_small.png').data, #w150, h68.58
            width='15%',
            height='15%'

            
        ),
        widgets.Box(layout=widgets.Layout(width='100px')),
        widgets.HTML(value="<h1>Naturgefahren-Info</h1>")
    ]),
    widgets.Box(
        layout=widgets.Layout(
            width='100px',
            height='10px', 
            background_color='transparent',
            padding = '0px',
            margin = '0px',
            border = 'none',
            border_bottom = 'none'
            )
    )
)


display(
    widgets.HTML(value=(
        "<div style='height:2px; width:100%; background-color:black; margin:0 auto;'></div><br>"+
        "<p>Untersuchen Sie die Risiken bezüglich Naturgefahren für eine Parzelle im Kanton Bern mittels Eingabe der Adresse <strong>oder</strong> der EGRID Nummer. "+ 
        "Die Gesamtfäche der Parzelle wird für die Informationsabfrage verwendet."+ 
        "<br>Datenstand: 11.10.2024</p>")
    )
)
display(
    widgets.VBox([
    widgets.HTML(value=
        "<h3>Gebäudeadresse</h3>"
        ),
    vbox
    ])
)
display(
    widgets.VBox([
    widgets.HTML(value=
        "<h3>EGRID Nummer</h3>"
        ),
    egrid
    ])
)

display(
    vbox_result
)

Box(layout=Layout(border='none', height='10px', margin='0px', padding='0px', width='100px'))

HTML(value="<div style='height:2px; width:100%; background-color:black; margin:0 auto;'></div><br><p>Untersuch…